In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
#Create lists for importing
SDUD_files = ['2018 State Drug Utilization Data.csv','2019 State Drug Utilization Data.csv',
              '2020 State Drug Utilization Data.csv', '2021 State Drug Utilization Data.csv',
              '2022 State Drug Utilization Data.csv']

NADAC_files = ['NADAC 2018.csv', '2019 NADAC.csv', '2020 NADAC.csv', '2021 NADAC.csv', '2022 NADAC.csv']

SAIPE_files = ['2018 SAIPE.csv', '2019 SAIPE.csv', '2020 SAIPE.csv', '2021 SAIPE.csv', '2022 SAIPE.csv']

data_files = ['States.csv', 'Unemployment by State.csv', 'CPI for all item.csv', 
              'Medicaid Enrollment Numbers Monthly.csv', 'GDP.csv', 'Mortgage Rates.csv'  ]

ndc_text_files = ['package.txt', 'product.txt']


dfs = ['states_df', 'unemp_df', 'cpi_df', 'benef_df', 'gdp_df', 'mortgage_df']

In [3]:
#Read in State Drug Utilization Dataset
SDUD =[]

for file in SDUD_files:
    df = pd.read_csv(file, dtype = {'ndc':str}, index_col = False)
    df.columns = df.columns.str.lower()
    SDUD.append(df)
    
SDUD_df = pd.concat(SDUD, ignore_index=True)

In [4]:
#Read in NADAC Dataset
NADAC = []

for file in NADAC_files:
    df = pd.read_csv(file, dtype ={'ndc': str},index_col = False)
    df.columns = df.columns.str.lower()
    NADAC.append(df)
    
NADAC_df = pd.concat(NADAC, ignore_index = True)

In [5]:
#Read in Saipe Dataset
SAIPE = []
year = 2018

for file in SAIPE_files:
    df = pd.read_csv(file, encoding='latin1', index_col = False)
    df.columns = df.columns.str.lower()
    df['year'] = year
    SAIPE.append(df)
    year += 1
    
saipe_df = pd.concat(SAIPE, ignore_index = True)

In [6]:
#read in all one-off files
for df_name, file in zip(dfs, data_files):
    # Read the CSV file and assign it to the DataFrame with the current name
    globals()[df_name] = pd.read_csv(file, index_col=False)
    

In [7]:
#Read in NDC dataset
ndc = []

for file in ndc_text_files:
    df = pd.read_csv(file, delimiter= '\t', encoding = 'latin-1', index_col=False)
    df.columns = df.columns.str.lower()
    ndc.append(df)
    
ndc_df = pd.merge(ndc[0], ndc[1], on = 'productid', how = 'left')
    

SDUD Cleaning

In [8]:
#drop records that have are missing total_amount_reimbursed
SDUD_df.dropna(subset = ['total_amount_reimbursed'], inplace = True)
#drop redundant columns related to NDC
SDUD_df.drop(columns = ['labeler_code', 'product_code', 'package_size'],inplace = True)

In [9]:
#get full state name on SDUD based on State Code and drop those without a state name bc that is national data
states_df.rename(columns = {'State Code':'state'}, inplace = True)
states_df['State Name'] = states_df['State Name'].str.strip()
combined_df = pd.merge(SDUD_df, states_df, on = 'state', how = 'left' )
combined_df.dropna(subset = ['State Name'], inplace = True)

In [10]:
#create unique variable for joining
combined_df['year-quarter'] = combined_df['year'].astype(str)+ 'Q' + combined_df['quarter'].astype(str)
#combined state and quarter
combined_df['state-quarter'] = combined_df['State Name'].astype(str) + combined_df['year-quarter'].astype(str)
#combined state and year
combined_df['state-year'] = combined_df['State Name'] + combined_df['year'].astype(str)
#drop state column
combined_df.drop(columns = ['state'], inplace = True)
#create date_ndc as unique variable consisting of state, quarter, and ndc for NADAC joining
combined_df['date_ndc'] = combined_df['ndc'].astype(str)+combined_df['state-quarter'].astype(str)


In [11]:
#Duplicate date_ndc combos based on utilization_type were combined, uncombining to see if there is an association
sum_columns = {
    'units_reimbursed': 'sum',
    'number_of_prescriptions': 'sum',
    'total_amount_reimbursed': 'sum',
    'medicaid_amount_reimbursed': 'sum',
    'non_medicaid_amount_reimbursed': 'sum'
}

SDUD_sum_df = combined_df.groupby('date_ndc').agg(sum_columns).reset_index()


In [12]:
#dropping and readding columns to the aggregated dataset
combin_drop = combined_df.copy()
columns_to_drop = ['utilization_type', 'suppression_used','units_reimbursed', 'number_of_prescriptions', 'total_amount_reimbursed', 'medicaid_amount_reimbursed', 'non_medicaid_amount_reimbursed']
combin_drop.drop(columns=columns_to_drop, inplace=True)
combin_drop =combin_drop.drop_duplicates()

In [13]:
#adding in the columns to the aggregated dataset
combined_df = pd.merge(SDUD_sum_df, combin_drop, on = 'date_ndc', how ='left')

In [14]:
#create date ndc combo for combination
combined_df['date_ndc_combo'] = combined_df['ndc'].astype(str)+combined_df['year-quarter'].astype(str)

NADAC Cleaning

In [15]:
#convert data to qqyyyy format and combine data with NDC fo combination
NADAC_df['effective_date'] = pd.to_datetime(NADAC_df['effective_date'])
NADAC_df['effective_date'] = NADAC_df['effective_date'].dt.to_period('Q')
NADAC_df['date_ndc_combo'] = NADAC_df['ndc'].astype(str)+NADAC_df['effective_date'].astype(str)

In [16]:
#calculated aveaege nadac_per_unit_ and corresponding_generic_drug_nadac_per_unit based on quarter
avg_df = NADAC_df.groupby(['date_ndc_combo'])[['nadac_per_unit', 'corresponding_generic_drug_nadac_per_unit']].mean().reset_index()
avg_df.rename(columns={'nadac_per_unit': 'quarteravg_nadac_per_unit', 'corresponding_generic_drug_nadac_per_unit': 'quarteravg_generic_nadac_per_unit'}, inplace=True)

In [17]:
#create the basis for the NADAC dataset. Since the reset of the fields are categorical and should not change for the same ndc
df_right_unique = NADAC_df.drop_duplicates(subset='date_ndc_combo')
NADAC_df = pd.merge(avg_df, df_right_unique, on = 'date_ndc_combo', how = 'inner')

In [18]:
#drop effective dates outside of the scope of the project
NADAC_df = NADAC_df[NADAC_df['effective_date'] <= '2022Q4']
NADAC_df = NADAC_df[NADAC_df['effective_date'] >= '2018Q1']

In [19]:
#drop additional columns that are not in use
NADAC_df.drop(columns = ['nadac_per_unit','effective_date','pharmacy_type_indicator','explanation_code','corresponding_generic_drug_nadac_per_unit', 'corresponding_generic_drug_effective_date', 'as_of_date', 'ndc', 'ndc description', 'as of date'], inplace = True)

NDC Data Cleaning

In [20]:
#drop duplicate columns from joining and rename
ndc_df.drop(columns = ['productndc_y', 'startmarketingdate_y', 'endmarketingdate_y','ndc_exclude_flag_y'],inplace= True)
ndc_df.rename(columns = {'productndc_x':'productndc', 'startmarketingdate_x':'startmarketingdate', 'endmarketingdate_x':'endmarketingdate', 'ndc_exclude_flag_x':'ndc_exclude_flag'}, inplace = True)


In [21]:
#drop columns that are duplicates of ndc or not necessary because of ndc
ndc_df.drop(columns=['productid', 'productndc'], inplace = True)

In [22]:
#drop duplicate rows
ndc_df = ndc_df.drop_duplicates()

In [23]:
#concatentate proprietary name and suffix columns so full name for drug in one column
ndc_df['proprietaryname'] = ndc_df['proprietaryname'] + ' ' + ndc_df['proprietarynamesuffix'].fillna(' ')


In [24]:
#drop unneccessary columns to analysis
ndc_df.drop(columns = ['proprietarynamesuffix','packagedescription', 'sample_package', 'endmarketingdate', 
                       'listing_record_certified_through', 'applicationnumber',
                       'active_numerator_strength', 'active_ingred_unit', 'ndc_exclude_flag'], inplace = True)

In [25]:
#convert marketing data to qqyyyy format
ndc_df['startmarketingdate'] = pd.to_datetime(ndc_df['startmarketingdate'])
ndc_df['startmarketingdate'] = ndc_df['startmarketingdate'].dt.to_period('Q')

In [26]:
#split ndc code into three separate columns on '-'
ndc_df[['label_code', 'product_code', 'package_code']] = ndc_df['ndcpackagecode'].str.split('-', expand=True)

In [27]:
#method converts 10-digit ndc to 11-digit ndc used for Medicaid billing
def ndc_conversion(row):
    if pd.isna(row['label_code']) or pd.isna(row['product_code']) or pd.isna(row['package_code']):
        return "Invalid NDC format"
    if len(row['label_code'])==4 and len(row['product_code'])==4 and len(row['package_code'])==2:
        return f"0{row['label_code']}{row['product_code']}{row['package_code']}"
    elif len(row['label_code'])==5 and len(row['product_code']) == 3 and len(row['package_code'])==2:
        return f"{row['label_code']}0{row['product_code']}{row['package_code']}"
    elif len(row['label_code'])==5 and len(row['product_code'])==4 and len(row['package_code'])==1:
        return f"{row['label_code']}{row['product_code']}0{row['package_code']}"
    else:
        return "Invalid Format"

In [28]:
#convert ndc code in ndc dataframe to 11-digit format
ndc_df['ndc'] = ndc_df.apply(ndc_conversion, axis =1)

CPI Data Cleaning

In [29]:
#drop series ID that is not seasonally adjusted so that the seasonally adjusted series is used
cpi_df.drop(cpi_df[cpi_df['Series ID'] =='CUUR0000SA0'].index, inplace = True)

In [30]:
cpi_df.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SA0,2018,M01,2018 Jan,248.859,0.4
1,CUSR0000SA0,2018,M02,2018 Feb,249.529,0.3
2,CUSR0000SA0,2018,M03,2018 Mar,249.577,0.0
3,CUSR0000SA0,2018,M04,2018 Apr,250.227,0.3
4,CUSR0000SA0,2018,M05,2018 May,250.792,0.2


In [31]:
#converrt to quarter and take the mean CPI for the quarter
cpi_df['Label'] = pd.to_datetime(cpi_df['Label'], format='%Y %b')
cpi_df['year-quarter'] = cpi_df['Label'].dt.to_period('Q')
cpi_quarter_avg = cpi_df.pivot_table(index='year-quarter', values='Value', aggfunc='mean').reset_index()
cpi_quarter_avg.rename(columns = {'Value':'CPI'}, inplace = True)
cpi_quarter_avg['year-quarter'] = cpi_quarter_avg['year-quarter'].astype(str)

Unemployment Data Cleaning

In [32]:
#rename series to state
unemp_df.rename(columns = {'Series ID':'State'}, inplace = True)

In [33]:
#Dictionary of series ID's and state names
replace_dict = {
    'LASST010000000000003': 'Alabama',
    'LASST020000000000003': 'Alaska',
    'LASST320000000000003': 'Nevada',
    'LASST330000000000003': 'New Hampshire',
    'LASST340000000000003': 'New Jersey',
    'LASST350000000000003': 'New Mexico',
    'LASST360000000000003': 'New York',
    'LASST370000000000003': 'North Carolina',
    'LASST380000000000003': 'North Dakota',
    'LASST390000000000003': 'Ohio',
    'LASST400000000000003': 'Oklahoma',
    'LASST410000000000003': 'Oregon',
    'LASST420000000000003': 'Pennsylvania',
    'LASST440000000000003': 'Rhode Island',
    'LASST450000000000003': 'South Carolina',
    'LASST460000000000003': 'South Dakota',
    'LASST470000000000003': 'Tennessee',
    'LASST480000000000003': 'Texas',
    'LASST490000000000003': 'Utah',
    'LASST500000000000003': 'Vermont',
    'LASST510000000000003': 'Virginia',
    'LASST530000000000003': 'Washington',
    'LASST540000000000003': 'West Virginia',
    'LASST550000000000003': 'Wisconsin',
    'LASST560000000000003': 'Wyoming',
    'LASST310000000000003': 'Nebraska',
    'LASST300000000000003': 'Montana',
    'LASST290000000000003': 'Missouri',
    'LASST160000000000003': 'Idaho',
    'LASST040000000000003': 'Arizona',
    'LASST050000000000003': 'Arkansas',
    'LASST060000000000003': 'California',
    'LASST080000000000003': 'Colorado', 
    'LASST090000000000003': 'Connecticut',  
    'LASST100000000000003': 'Delaware',
    'LASST110000000000003': 'Washington DC',  
    'LASST120000000000003': 'Florida',
    'LASST130000000000003': 'Georgia',   
    'LASST150000000000003': 'Hawaii',   
    'LASST170000000000003': 'Illinois',  
    'LASST280000000000003': 'Mississippi',
    'LASST180000000000003': 'Indiana',  
    'LASST190000000000003': 'Iowa', 
    'LASST200000000000003': 'Kansas',   
    'LASST210000000000003': 'Kentucky',   
    'LASST220000000000003': 'Louisiana',   
    'LASST230000000000003': 'Maine',   
    'LASST240000000000003': 'Maryland',   
    'LASST250000000000003': 'Massachusetts',   
    'LASST260000000000003': 'Michigan',  
    'LASST270000000000003': 'Minnesota',   
}

In [34]:
#replace Series ID with state name and drop the Puerto Rico Series ID
unemp_df['State'] = unemp_df['State'].replace(replace_dict)
unemp_df.drop(unemp_df[unemp_df['State']=='LASST720000000000003'].index, inplace = True)

In [35]:
#convert to date time and create state quarter variable
unemp_df['Label'] = pd.to_datetime(unemp_df['Label'], format='%Y %b')
unemp_df['Month'] = unemp_df['Label'].dt.to_period('m').dt.month
unemp_df['Quarter'] = unemp_df['Label'].dt.strftime('%Y') + 'Q' + unemp_df['Label'].dt.quarter.astype(str)
unemp_df['state-quarter'] = unemp_df['State'].astype(str) + unemp_df['Quarter'].astype(str)

In [36]:
#drop columns and rename unemp_rate
unemp_df.drop(columns = ['Year', 'Period', 'Label','State','Month'], inplace = True)
unemp_df.rename(columns= {'Value':'unemp_rate'}, inplace = True)

In [37]:
#convert unemp_rate to a number
unemp_df['unemp_rate'] = pd.to_numeric(unemp_df['unemp_rate'], errors='coerce')

In [38]:
#get average of the unemployment rate by quarter
unemp_quarter_avg_df = unemp_df.pivot_table(index= 'state-quarter', values='unemp_rate', aggfunc='mean').reset_index()

Beneficiary Data Cleaning

In [39]:
#drop program types that are not Medicaid
benef_df = benef_df.drop(benef_df[benef_df['ProgramType'] != 'Medicaid'].index)
#convert countenrolled to an int
benef_df['CountEnrolled'] = benef_df['CountEnrolled'].str.replace(',', '').astype(int)
#drop DQUnusable that the Unusable or High Concern
benef_df['DQUnusable'] = benef_df['DQUnusable'].str.strip()
benef_df = benef_df.drop(benef_df[benef_df['DQUnusable']== 'Unusable'].index)
benef_df = benef_df.drop(benef_df[benef_df['DQUnusable']== 'High Concern'].index)
#convert month and quarter
benef_df['Month'] = pd.to_datetime(benef_df['Month'], format = '%Y%m')
benef_df['Quarter'] = benef_df['Month'].dt.to_period('Q')
#create state-quarter variable for combination
benef_df['state-quarter'] = benef_df['State'] + benef_df['Quarter'].astype(str)
benef_df.drop(columns = ['DQUnusable', 'State', 'Month'], inplace = True)

In [40]:
benef_avg_df = benef_df.groupby('state-quarter')['CountEnrolled'].mean().reset_index()

SAIPE Data Cleaning

In [41]:
saipe_df.rename(columns = {'name': 'state name'}, inplace = True)

In [42]:
#drop records that are counties 
state_names= states_df['State Name']
indices_to_drop = saipe_df[~saipe_df['state name'].isin(state_names)].index
saipe_df = saipe_df.drop(indices_to_drop)

In [43]:
saipe_df['state-year'] = saipe_df['state name'] + saipe_df['year'].astype(str)

In [44]:
#drop columns that are not relevant to analysis
columns_to_drop = [col for col in saipe_df.columns if col.startswith('90%')]
saipe_df = saipe_df.drop(columns=columns_to_drop)

columns_to_drop = [col for col in saipe_df.columns if col.startswith('unnamed')]
saipe_df = saipe_df.drop(columns=columns_to_drop)

saipe_df.drop(columns = ['ï»¿state fips code', 
                        'county fips code', 'postal code', 'poverty estimate, age 0-17', 
                        'poverty percent, age 0-17', 'poverty estimate, age 5-17 in families',
                        'poverty percent, age 5-17 in families', 'poverty estimate, age 0-4',
                        'poverty percent, age 0-4', 'state name', 'year'], inplace = True)

In [45]:
#convert median household income and poverty estimate to numeric
saipe_df['median household income'] = saipe_df['median household income'].str.replace(',', '', regex=False).astype(float)
saipe_df['poverty estimate, all ages'] = saipe_df['poverty estimate, all ages'].str.replace('[^\d.]', '', regex=True).astype(float)

Mortgage Rates

In [46]:
#convert date and create year-quarter variable
mortgage_df['DATE'] = pd.to_datetime(mortgage_df['DATE'],format='%m/%d/%y')
mortgage_df['year-quarter'] = mortgage_df['DATE'].dt.to_period('Q')
mortgage_df.drop(columns = {'DATE'}, inplace = True)

In [47]:
#get average mortgage rate for the quarter
mortgage_df_pivot = mortgage_df.pivot_table(index = 'year-quarter', values = 'MORTGAGE30US', aggfunc = 'mean').reset_index()
mortgage_df_pivot['year-quarter'] = mortgage_df_pivot['year-quarter'].astype(str)

Dataset Assembly

In [48]:
#combine SDUD dataset with unempoyment rate
combined_df = pd.merge(combined_df, unemp_quarter_avg_df, on = 'state-quarter', how = 'left')

In [49]:
#combine SDUD dataset with beneficiary dataset
combined_df = pd.merge(combined_df, benef_avg_df, on = 'state-quarter', how = 'left')

In [50]:
#combine SDUD dataset with SAIPE dataset
combined_df = pd.merge(combined_df, saipe_df, on= 'state-year', how = 'left')

In [51]:
#combine SDUD dataset with NADAC dataset
combined_df = pd.merge(combined_df,NADAC_df, on = 'date_ndc_combo', how = 'left')

In [52]:
#combine SDUD dataset with ndc dataset
combined_df = pd.merge(combined_df, ndc_df, on = 'ndc', how = 'left')

In [53]:
#combine SDUD dataset with CPI dataset
combined_df = pd.merge(combined_df, cpi_quarter_avg, on='year-quarter', how='left')

In [54]:
#combine SDUD with GDP datset
combined_df = pd.merge(combined_df, gdp_df, on='year-quarter', how='left')

In [55]:
#combine SDUD with mortgage
df = pd.merge(combined_df, mortgage_df_pivot, on ='year-quarter', how = 'left')

In [56]:
#convert GDP to flaot
df['GDP in billions'] = combined_df['GDP in billions'].str.replace(',', '', regex=False).astype(float)

Cleaning

In [57]:
#drop unneeded columns
df.drop(columns =['date_ndc', 'state-quarter', 'date_ndc_combo',
                  'label_code', 'product_code', 'package_code', 'routename', 'otc', 'pricing_unit'], inplace = True)

In [58]:
#drop unneeded columns
df.drop(columns = ['ndc_description','quarteravg_generic_nadac_per_unit','non_medicaid_amount_reimbursed',
                  'medicaid_amount_reimbursed', 'startmarketingdate','nonproprietaryname',
                   'ndc', 'quarter']
                  , inplace = True)

In [59]:
#df['year'] = df['year'].astype(str)
df['year-quarter'] = pd.to_datetime(df['year-quarter'])
df['year-quarter'] = pd.to_datetime(df['year-quarter']).dt.to_period('Q')
df['poverty percent, all ages'] = df['poverty percent, all ages'].astype(float)

/var/folders/lf/2w97dh191txdbq0mypdjfc7w0000gn/T/ipykernel_2952/2132680326.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['year-quarter'] = pd.to_datetime(df['year-quarter'])


In [60]:
#drop records where ndcpackagecode is null
df_cleaned = df[df['ndcpackagecode'].notna()]


In [61]:
#drop records where poverty estimate is null
df_cleaned = df_cleaned[df_cleaned['poverty estimate, all ages'].notna()]

In [62]:
#get rid of total amounts reimbursed that less than $1
df_cleaned = df_cleaned[df_cleaned['total_amount_reimbursed'] > 1.00]

In [63]:
ndc_counts = df_cleaned['ndcpackagecode'].value_counts()

# Filter the DataFrame to keep only records with ndcpackagecode count >= 100
df_cleaned= df_cleaned[df_cleaned['ndcpackagecode'].isin(ndc_counts[ndc_counts >= 100].index)]

In [64]:
#B-ANDA is a generic for a drug previously approved as a brand name, B-ANDA will be replaced with G (generic)
df_cleaned['classification_for_rate_setting'] = df_cleaned['classification_for_rate_setting'].replace('B-ANDA', 'G')

In [65]:
df_no_imputations = df_cleaned.copy()

In [69]:
def fill_missing_with_mode(group):
    mode = group.mode()
    if not mode.empty:
        return mode.iloc[0]
    else:
        return None

# Group by 'product_name' then apply the function to fill missing values
mode_by_group = df_cleaned.groupby(['product_name'])['classification_for_rate_setting'].apply(fill_missing_with_mode)

# Fill missing values in 'classification_for_rate_setting' with the mode for each group
df_cleaned['classification_for_rate_setting'] = df_cleaned.apply(lambda row: mode_by_group[row['product_name']]
                                              if pd.isna(row['classification_for_rate_setting']) and row['product_name'] in mode_by_group
                                              else row['classification_for_rate_setting'], axis=1)

In [70]:
df_cleaned['deaschedule'] = df_cleaned.apply(lambda row: 'Not Controlled'
                                            if pd.isna(row['deaschedule'])
                                            else row['deaschedule'], axis = 1)

In [71]:
df_cleaned.drop(columns = ['substancename'], inplace = True)

In [72]:
average_prices = df_cleaned.groupby(['product_name', 'year'])['quarteravg_nadac_per_unit'].mean()

# Fill NaN values with the corresponding average price
df_cleaned['quarteravg_nadac_per_unit'] = df_cleaned.apply(lambda row: average_prices[row['product_name'], row['year']]
                                        if pd.isna(row['quarteravg_nadac_per_unit']) else row['quarteravg_nadac_per_unit'], axis=1)

In [73]:
average_prices = df_cleaned.groupby(['product_name'])['quarteravg_nadac_per_unit'].mean()

# Fill NaN values with the corresponding average price
df_cleaned['quarteravg_nadac_per_unit'] = df_cleaned.apply(lambda row: average_prices[row['product_name']]
                                        if pd.isna(row['quarteravg_nadac_per_unit']) else row['quarteravg_nadac_per_unit'], axis=1)

In [74]:
average_unemp = df_cleaned.groupby(['state-year'])['unemp_rate'].mean()

# Fill NaN values with the corresponding average price
df_cleaned['unemp_rate'] = df_cleaned.apply(lambda row: average_unemp[row['state-year']]
                                        if pd.isna(row['unemp_rate']) else row['unemp_rate'], axis=1)

In [75]:
df_cleaned = df_cleaned[df_cleaned['quarteravg_nadac_per_unit'].notna()]

In [76]:
average_enrolled = df_cleaned.groupby(['State Name'])['year'].mean()

# Fill NaN values with the corresponding average price
df_cleaned['CountEnrolled'] = df_cleaned.apply(lambda row: average_enrolled[row['State Name']]
                                        if pd.isna(row['CountEnrolled']) else row['CountEnrolled'], axis=1)


/var/folders/lf/2w97dh191txdbq0mypdjfc7w0000gn/T/ipykernel_2149/4117733354.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['CountEnrolled'] = df_cleaned.apply(lambda row: average_enrolled[row['State Name']]


In [77]:
#split the dosage name keeping only the first instance since remainder of the string provides extraneous information
df_cleaned['dosageformname'] = df_cleaned['dosageformname'].str.split(',').str[0]

/var/folders/lf/2w97dh191txdbq0mypdjfc7w0000gn/T/ipykernel_2149/3685131298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['dosageformname'] = df_cleaned['dosageformname'].str.split(',').str[0]


In [78]:
# Step 1: Calculate lower and upper bounds for outliers
Q1 = df_cleaned[['units_reimbursed', 'total_amount_reimbursed', 'number_of_prescriptions', 'quarteravg_nadac_per_unit']].quantile(0.25)
Q3 = df_cleaned[['units_reimbursed', 'total_amount_reimbursed', 'number_of_prescriptions', 'quarteravg_nadac_per_unit']].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

# Step 2: Filter the original DataFrame to keep only rows without outliers
df_no_outliers = df_cleaned[
    (df_cleaned['units_reimbursed'] >= lower_bound['units_reimbursed']) &
    (df_cleaned['units_reimbursed'] <= upper_bound['units_reimbursed']) &
    (df_cleaned['total_amount_reimbursed'] >= lower_bound['total_amount_reimbursed']) &
    (df_cleaned['total_amount_reimbursed'] <= upper_bound['total_amount_reimbursed']) &
    (df_cleaned['number_of_prescriptions'] >= lower_bound['number_of_prescriptions']) &
    (df_cleaned['number_of_prescriptions'] <= upper_bound['number_of_prescriptions'])&
    (df_cleaned['quarteravg_nadac_per_unit'] >= lower_bound['quarteravg_nadac_per_unit']) &
    (df_cleaned['quarteravg_nadac_per_unit'] <= upper_bound['quarteravg_nadac_per_unit'])
]

In [79]:
df_no_outliers.shape

(5142457, 27)

In [80]:
df_no_outliers.drop(columns= ['product_name'],inplace = True)

/var/folders/lf/2w97dh191txdbq0mypdjfc7w0000gn/T/ipykernel_2149/3685220815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outliers.drop(columns= ['product_name'],inplace = True)


Pharm Classes

In [81]:
pharm_df = ndc_df[['ndcpackagecode', 'pharm_classes']].copy()

In [82]:
pharm_df.dropna(subset=['pharm_classes'], inplace=True)

In [83]:
pharm_df['pharm_classes']= pharm_df['pharm_classes'].astype(str)

In [84]:
def clean_classes(text):
    # Use regular expression to remove square brackets and their contents
    cleaned_text = re.sub(r'\[.*?\]', '', text)
    # Remove leading and trailing whitespace
    cleaned_text = cleaned_text.strip()
    return cleaned_text

# Clean 'pharm_classes' column
pharm_df['pharm_classes'] = pharm_df['pharm_classes'].apply(clean_classes)

# Split the 'pharm_classes' column by comma and strip whitespace
pharm_df['pharm_classes'] = pharm_df['pharm_classes'].str.split(',').apply(lambda x: [item.strip() for item in x])

# Remove duplicates from each list of classes
pharm_df['pharm_classes'] = pharm_df['pharm_classes'].apply(lambda x: list(set(x[:5])))

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform the 'pharm_classes' column into binary columns
binary_matrix = mlb.fit_transform(pharm_df['pharm_classes'])

# Access feature names
feature_names = mlb.classes_

# Create DataFrame with binary columns
binary_df = pd.DataFrame(binary_matrix, columns=feature_names)

# Concatenate binary DataFrame with the original DataFrame
pharm_combined_df = pd.concat([pharm_df, binary_df], axis=1)


In [85]:
pharm_combined_df.head()

,ndcpackagecode,pharm_classes,4-Hydroxyphenyl-Pyruvate Dioxygenase Inhibitor,5-Lipoxygenase Inhibitor,5-Lipoxygenase Inhibitors,5-alpha Reductase Inhibitor,5-alpha Reductase Inhibitors,AMPA Receptor Antagonists,Acetyl Aldehyde Dehydrogenase Inhibitors,Acetylcholine Release Inhibitor,Acetylcholine Release Inhibitors,Acidifying Activity,Actinomycin,Actively Acquired Immunity,Adenosine Receptor Agonist,Adenosine Receptor Agonists,Adenosine Triphosphate-Citrate Lyase Inhibitor,Adenosine Triphosphate-Citrate Lyase Inhibitors,Adenovirus,Adrenal Steroid Synthesis Inhibitor,Adrenal Steroid Synthesis Inhibitors,Adrenergic Agonists,Adrenergic Receptor Agonist,Adrenergic alpha-Agonists,Adrenergic alpha-Antagonists,Adrenergic alpha1-Agonists,Adrenergic alpha2-Agonists,Adrenergic beta-Agonists,Adrenergic beta-Antagonists,Adrenergic beta1-Antagonists,Adrenergic beta2-Agonists,Adrenergic beta2-Antagonists,Adrenergic beta3-Agonists,Adrenocorticotropic Hormone,Aldehyde Dehydrogenase Inhibitor,Aldosterone Antagonist,Aldosterone Antagonists,Alkaline Phosphatase,Alkalinizing Activity,Alkaloid,Alkaloids,Alkylating Activity,Alkylating Drug,Allergens,Allogeneic,Allogeneic Cord Blood Hematopoietic Progenitor Cell Therapy,Allogeneic Cultured Cell Scaffold,Allylamine,Allylamine Antifungal,Aluminum Complex,Alveolar Surface Tension Reduction,Amide Local Anesthetic,Amides,Amino Acid,Amino Acid Hypertonic Solution,Amino Acids,Aminoglycoside Antibacterial,Aminoglycosides,Aminoketone,Aminosalicylate,Aminosalicylic Acids,Ammonium Ion Binding Activity,Amphenicol-class Antibacterial,Amphenicols,Amphetamine Anorectic,Amphetamines,Amyloid Beta-directed Antibody,Amyloid Beta-directed Antibody Interactions,Analogs/Derivatives,Androgen,Androgen Receptor Agonists,Androgen Receptor Antagonists,Androgen Receptor Inhibitor,Androstanes,Angiopoietin-like 3 Inhibitor,Angiopoietin-like 3 Inhibitors,Angiotensin 2 Receptor Antagonists,Angiotensin 2 Receptor Blocker,Angiotensin 2 Type 1 Receptor Antagonists,Angiotensin Converting Enzyme Inhibitor,Angiotensin-converting Enzyme Inhibitors,Animal Fur,Ant Venoms,Anthelmintic,Anthelmintics,Anthracycline Topoisomerase Inhibitor,Anthracyclines,Anthrax Protective Antigen-directed Antibody,Anthrax Protective Antigen-directed Antibody Interactions,Anthrax Vaccines,Anti-IgE,Anti-Inflammatory Agents,Anti-anginal,Anti-coagulant,Anti-epileptic Agent,Antiarrhythmic,Antibodies,Antibody-Surface Protein Interactions,Anticholinergic,Antidiarrheal,Antidote,Antidote for Acetaminophen Overdose,Antiemetic,Antifibrinolytic Agent,Antigen Neutralization,Antigens,Antihistamine,Antihypoglycemic Agent,Antileishmanial,Antimalarial,Antimetabolite,Antimetabolite Immunosuppressant,Antimycobacterial,Antiparasitic,Antiprotozoal,Antirheumatic Agent,Antisense,Antisense Oligonucleotide,Antitoxins,Antivenin,Antivenins,Appetite Suppression,Aromatase Inhibitor,Aromatase Inhibitors,Aromatic,Aromatic Amino Acid,Arteriolar Vasodilation,Arteriolar Vasodilator,Aryl Hydrocarbon Receptor Agonist,Aryl Hydrocarbon Receptor Agonists,Ascorbic Acid,Asparaginase,Asparagine-specific Enzyme,Attenuated,Atypical Antipsychotic,Autologous,Autologous Cellular Immunotherapy,Autologous Cultured Cell,Autonomic Ganglionic Blocker,Azole Antifungal,Azoles,B Lymphocyte Stimulator-directed Antibody Interactions,B Lymphocyte Stimulator-specific Inhibitor,BCG Vaccine,BCL-2 Inhibitor,Bacterial,Bacterial Neurotoxin Neutralization,Barbiturate,Barbiturates,Bcr-Abl Tyrosine Kinase Inhibitors,Bee Venoms,Benzodiazepine,Benzodiazepine Antagonist,Benzodiazepines,Benzothiazole,Benzothiazoles,Benzylamine Antifungal,Benzylamines,Biguanide,Biguanides,Bile Acid,Bile Acid Sequestrant,Bile Acids and Salts,Bile Salt Export Pump Inhibitors,Bile-acid Binding Activity,Bismuth,Bisphosphonate,Blood Coagulation Factor,Blood Coagulation Factors,Blood Viscosity Reducer,Boron Compounds,Bradykinin B2 Receptor Antagonist,Bradykinin B2 Receptor Antagonists,Breast Cancer Resistance Protein Inhibitors,Bruton's Tyrosine Kinase 

In [86]:
pharm_combined_df.drop(columns = ['pharm_classes'], inplace = True)

In [87]:
numerical_columns = pharm_combined_df.select_dtypes(include=['number']).columns

# Calculate the sum of each numerical column
column_sums = pharm_combined_df[numerical_columns].sum()

# Filter out columns where the sum is less than 200
columns_to_drop = column_sums[column_sums < 200].index

# Drop the filtered columns from the DataFrame
pharm_df_combined = pharm_combined_df.drop(columns=columns_to_drop)

In [88]:
pharm_combined_df.fillna(0, inplace=True)

In [89]:
pharm_combined_df['ndcpackagecode'] = pharm_combined_df['ndcpackagecode'].astype(str)

# Set all other columns to integer data type
pharm_combined_df.iloc[:, 1:] = pharm_combined_df.iloc[:, 1:].astype(int)

In [90]:
pharm_combined_df.drop_duplicates(subset='ndcpackagecode', inplace=True)

In [91]:
# Drop duplicates based on 'ndcpackagecode' column
pharm_combined_df.drop_duplicates(subset=['ndcpackagecode'], inplace=True)


Join Pharm Classes

In [92]:
pharm_ndc_df = ndc_df[['ndcpackagecode']].copy()

In [93]:
final_ndc_pharm_df = pd.merge(pharm_ndc_df, pharm_combined_df, on = 'ndcpackagecode', how = 'left')

In [94]:
pharm_ndc_df.drop_duplicates(subset='ndcpackagecode', inplace=True)

In [95]:
final_ndc_pharm_df.dropna(inplace=True)


In [96]:
numerical_columns = final_ndc_pharm_df.select_dtypes(include=['number']).columns

# Calculate the sum of each numerical column
column_sums = final_ndc_pharm_df[numerical_columns].sum()

In [97]:
percentile_90 = column_sums.quantile(0.90)

# Identify columns with sums below the 90th percentile
low_sum_columns = column_sums[column_sums < percentile_90].index

# Drop the identified columns from the DataFrame
df_filtered = final_ndc_pharm_df.drop(columns=low_sum_columns)

In [98]:
df_imputations = pd.merge(df_no_outliers,df_filtered, on = 'ndcpackagecode', how = 'left')

In [99]:
columns_to_fill = df_filtered.columns.tolist()

# Replace NaN values with 0 in specified columns
df_imputations[columns_to_fill] = df_imputations[columns_to_fill].fillna(0)

In [100]:
df_imputations['Atypical Antipsychotic'].value_counts()

Atypical Antipsychotic
0.0    5068357
1.0      74100
Name: count, dtype: int64

In [101]:
numerical_columns = df_imputations.select_dtypes(include='number').columns

for i in numerical_columns:
    print(i, ': ', df_imputations[i].sum())

units_reimbursed :  33490436874.629025
number_of_prescriptions :  696683401.0
total_amount_reimbursed :  13043430938.600002
year :  10388629019
unemp_rate :  24072782.033333346
CountEnrolled :  9584937024175.443
poverty estimate, all ages :  4817348173842.0
poverty percent, all ages :  63873610.69999996
median household income :  348012850567.0
quarteravg_nadac_per_unit :  1895451.3126124013
CPI :  1375406723.8856657
GDP in billions :  117111372848884.0
MORTGAGE30US :  20445916.628736276
Adrenergic alpha-Agonists :  32614.0
Adrenergic alpha-Antagonists :  40861.0
Adrenergic alpha1-Agonists :  75536.0
Adrenergic alpha2-Agonists :  9453.0
Adrenergic beta-Antagonists :  73096.0
Allergens :  281281.0
Amide Local Anesthetic :  24704.0
Amides :  22996.0
Angiotensin 2 Receptor Antagonists :  50402.0
Angiotensin 2 Receptor Blocker :  50402.0
Angiotensin Converting Enzyme Inhibitor :  42761.0
Angiotensin-converting Enzyme Inhibitors :  42761.0
Anti-Inflammatory Agents :  208617.0
Anti-epileptic

In [102]:
df_imputations.columns.get_loc('Adrenergic alpha-Agonists')


26

In [103]:
flag_columns = df_imputations.columns[28:]
sums = df_imputations[flag_columns].sum()

# Identify columns where the sum is less than 30,000
columns_to_drop = sums[sums < 70000].index

# Drop columns from the DataFrame
df_imputations.drop(columns=columns_to_drop, inplace=True)

In [104]:
df_imputations.shape

(5142457, 61)

In [105]:
df_imputations['Atypical Antipsychotic'].value_counts()

Atypical Antipsychotic
0.0    5068357
1.0      74100
Name: count, dtype: int64

In [106]:
df_imputations.drop(columns = ['pharm_classes'],inplace = True)

In [107]:
df_imputations.isna().sum()

units_reimbursed                                                     0
number_of_prescriptions                                              0
total_amount_reimbursed                                              0
year                                                                 0
State Name                                                           0
Region                                                               0
year-quarter                                                         0
state-year                                                           0
unemp_rate                                                           0
CountEnrolled                                                        0
poverty estimate, all ages                                           0
poverty percent, all ages                                            0
median household income                                              0
quarteravg_nadac_per_unit                                            0
classi

## Dropping anything with nulls for the dataset with no imputation then dropping extraneous columns and adding pharm class work

In [66]:
df_no_imputations.isna().sum()

units_reimbursed                         0
number_of_prescriptions                  0
total_amount_reimbursed                  0
year                                     0
product_name                             0
State Name                               0
Region                                   0
year-quarter                             0
state-year                               0
unemp_rate                               0
CountEnrolled                       120632
poverty estimate, all ages               0
poverty percent, all ages                0
median household income                  0
quarteravg_nadac_per_unit          1970479
classification_for_rate_setting    1970479
ndcpackagecode                           0
producttypename                          0
proprietaryname                          0
dosageformname                           0
marketingcategoryname                    0
labelername                              0
substancename                       140841
pharm_class

In [67]:
df_no_imputations.dropna(subset = ['unemp_rate', 'CountEnrolled','quarteravg_nadac_per_unit',
                                         'classification_for_rate_setting','proprietaryname'],inplace=True)

In [68]:
df_no_imputations.drop(columns = ['pharm_classes','substancename'],inplace = True)

In [72]:
# Define a function to calculate outliers using the IQR method
numeric_columns = df_no_imputations.select_dtypes(include=[np.number])

# Define a function to calculate outliers using the IQR method
def count_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (column < lower_bound) | (column > upper_bound)
    return outliers.sum()
    
# Iterate over each column in the DataFrame and calculate outliers
outliers_count = {}
for column in numeric_columns.columns:
    outliers_count[column] = count_outliers(numeric_columns[column])

# Print the number of outliers for each numeric variable
for column, count in outliers_count.items():
    print(f"Variable '{column}': {count} outliers")


Variable 'units_reimbursed': 711307 outliers
Variable 'number_of_prescriptions': 693506 outliers
Variable 'total_amount_reimbursed': 691015 outliers
Variable 'year': 0 outliers
Variable 'unemp_rate': 499578 outliers
Variable 'CountEnrolled': 480927 outliers
Variable 'poverty estimate, all ages': 355826 outliers
Variable 'poverty percent, all ages': 142118 outliers
Variable 'median household income': 0 outliers
Variable 'quarteravg_nadac_per_unit': 773642 outliers
Variable 'CPI': 0 outliers
Variable 'GDP in billions': 0 outliers
Variable 'MORTGAGE30US': 0 outliers


In [111]:
# Step 1: Calculate lower and upper bounds for outliers
Q1 = df_no_imputations[['units_reimbursed', 'total_amount_reimbursed', 'number_of_prescriptions', 'quarteravg_nadac_per_unit']].quantile(0.25)
Q3 = df_no_imputations[['units_reimbursed', 'total_amount_reimbursed', 'number_of_prescriptions', 'quarteravg_nadac_per_unit']].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

# Step 2: Filter the original DataFrame to keep only rows without outliers
df_no_imputations = df_no_imputations[
    (df_no_imputations['units_reimbursed'] >= lower_bound['units_reimbursed']) &
    (df_no_imputations['units_reimbursed'] <= upper_bound['units_reimbursed']) &
    (df_no_imputations['total_amount_reimbursed'] >= lower_bound['total_amount_reimbursed']) &
    (df_no_imputations['total_amount_reimbursed'] <= upper_bound['total_amount_reimbursed']) &
    (df_no_imputations['number_of_prescriptions'] >= lower_bound['number_of_prescriptions']) &
    (df_no_imputations['number_of_prescriptions'] <= upper_bound['number_of_prescriptions'])&
    (df_no_imputations['quarteravg_nadac_per_unit'] >= lower_bound['quarteravg_nadac_per_unit']) &
    (df_no_imputations['quarteravg_nadac_per_unit'] <= upper_bound['quarteravg_nadac_per_unit'])
]

In [112]:
df_no_imputations = pd.merge(df_no_imputations,df_filtered, on = 'ndcpackagecode', how = 'left')

In [113]:
columns_to_fill = df_filtered.columns.tolist()

# Replace NaN values with 0 in specified columns
df_no_imputations[columns_to_fill] = df_no_imputations[columns_to_fill].fillna(0)

In [114]:
column_name = 'Adrenergic alpha-Agonists'

# Get the index of the column
index_of_column = df_no_imputations.columns.get_loc(column_name)

print("Index of column '{}' is: {}".format(column_name, index_of_column))

Index of column 'Adrenergic alpha-Agonists' is: 26


In [115]:
"""flag_columns = df_no_imputations.columns[26:]
sums = df_no_imputations[flag_columns].sum()

# Identify columns where the sum is less than 30,000
columns_to_drop = sums[sums < 70000].index"""

# Drop columns from the DataFrame
df_no_imputations.drop(columns=columns_to_drop, inplace=True)

In [116]:
df_no_imputations['deaschedule'] = df_no_imputations.apply(lambda row: 'Not Controlled'
                                            if pd.isna(row['deaschedule'])
                                            else row['deaschedule'], axis = 1)

In [117]:
df_no_imputations.drop(columns = ['product_name'], inplace = True)

In [119]:
ndc_counts = df_no_imputations['ndcpackagecode'].value_counts()

# Filter the DataFrame to keep only records with ndcpackagecode count >= 100
df_no_imputations= df_no_imputations[df_no_imputations['ndcpackagecode'].isin(ndc_counts[ndc_counts >= 100].index)]

Write CSV

In [116]:
df_imputations.shape

(5142457, 60)

In [117]:
for col in df_imputations.columns[28:]:
    df_imputations[col] = df_imputations[col].astype(int, errors='ignore')

In [119]:
df_imputations.to_csv('Medicaid Dataset with Imputations.csv')

In [120]:
df_no_imputations.shape

(3974522, 60)

In [121]:
df_no_imputations.isna().sum()

units_reimbursed                                                     0
number_of_prescriptions                                              0
total_amount_reimbursed                                              0
year                                                                 0
State Name                                                           0
Region                                                               0
year-quarter                                                         0
state-year                                                           0
unemp_rate                                                           0
CountEnrolled                                                        0
poverty estimate, all ages                                           0
poverty percent, all ages                                            0
median household income                                              0
quarteravg_nadac_per_unit                                            0
classi

In [122]:
df_no_imputations.to_csv('Medicaid Dataset with No Imputations.csv')